# Adaptive RAG

![Adaptive RAG 워크플로우](AdaptiveRAG.png)

**Adaptive RAG**는 질문의 특성에 따라 적절한 검색 전략을 동적으로 선택하는 고급 RAG 시스템입니다.

### 핵심 구성요소 
 **Query Classifier (쿼리 분류기)**- 사용자 질문을 분석하여 3가지 카테고리로 분류:  
    - `vectorstore`: 내부 문서 검색 필요 (예: LangChain, LangGraph 기술 문서)  
    - `web_search`: 웹 검색 필요 (예: 최신 뉴스, 실시간 정보)  
    - `direct_answer`: 검색 불필요 (예: 일반 상식, 수학 계산)   
 **Router (라우터)**- 분류 결과에 따라 적절한 노드로 라우팅- 조건부 엣지를 통한 동적 경로 결정    
 **Retriever Components (검색 컴포넌트)**   
    - **문서 Retriever**: Vector Store에서 관련 문서 검색   
    - **Web Search**: 외부 웹 검색 수행  
    - **Direct Answer**: 검색 없이 직접 답변 생성    

### 워크플로우

    ```
    사용자 질문    
    ↓[Classify Query] ← LLM으로 질문 분류    
    ↓[Route Query] ← 조건부 라우팅    
    ├─→ [Retrieve 문서s] → [Generate Answer]    
    ├─→ [Web Search] → [Generate Answer]    
    └─→ [Generate Answer] (직접)
    ```

## Setup

Load and/or check for needed environmental variables

In [ ]:
import uuid
import os
import nest_asyncio
nest_asyncio.apply() # 중첩 evnet loop 허용
from typing import Literal, TypedDict
from IPython.display import Image, display
from langchain_core.runnables.graph import  MermaidDrawMethod
from dotenv import load_dotenv

load_dotenv()

## VectorStore & Retriever

In [ ]:
"""
문서 포맷팅
"""
from typing import List
from langchain_core.documents import Document

def format_docs(docs: List[Document]) -> str:
    if not docs:
        return "문서를 찾을 수 없습니다."

    formatted = []
    for i, doc in enumerate(docs, 1):
        snippet = doc.page_content
        source = doc.metadata.get("source", "알 수 없음")
        # NOTE: Tip: **문서 간의 구분을 짓는 포맷팅은 굉장히 중요합니다.**
        formatted_line = f"[문서-{i}] (출처: {source}): " + snippet
        formatted.append(formatted_line)
    
    # NOTE: Tip: **문서 간의 구분을 짓는 포맷팅은 굉장히 중요합니다.**
    return "\n---\n".join(formatted)


def format_docs_detailed(docs: List[Document], max_length: int = 1000) -> str:
    if not docs:
        return "문서를 찾을 수 없습니다."

    formatted = []
    for i, doc in enumerate(docs, 1):
        snippet = doc.page_content[:max_length]
        metadata = ", ".join([f"{k}={v}" for k, v in doc.metadata.items()])
        formatted_line = f"<문서-{i} | 정보: {metadata} | 내용: {snippet}>"
        formatted.append(formatted_line)

    return "\n---\n".join(formatted)


In [ ]:
"""
벡터스토어 설정 (Qdrant 기반)
"""
from typing import List, Optional
# LangChain OpenAI 임베딩 모델
from langchain_openai import OpenAIEmbeddings
# Qdrant 벡터스토어 사용
from langchain_core.vectorstores import VectorStore
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
# 임베딩 캐싱을 위한 LangChain-Classic(V1.0 이하) 사용
from langchain_classic.embeddings import CacheBackedEmbeddings
from langchain_classic.storage import LocalFileStore
# 환경변수 로드
from dotenv import load_dotenv

load_dotenv()

def setup_vectorstore(
    documents: List,
    embeddings: Optional[OpenAIEmbeddings] = None,
    collection_name: str = "default",
) -> VectorStore:
    """Qdrant 벡터스토어를 생성하고 채웁니다 (in-memory)."""
    if embeddings is None:
        # 기본 임베딩 모델 생성
        underlying_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

        # 로컬 파일 기반 캐시 스토어 생성
        store = LocalFileStore("./cache/embeddings/")

        # CacheBackedEmbeddings로 감싸서 캐싱 활성화
        embeddings = CacheBackedEmbeddings.from_bytes_store(
            underlying_embeddings, 
            store, 
            namespace=underlying_embeddings.model,
        )

    # Qdrant client (in-memory)
    qdrant_client = QdrantClient(":memory:")

    # Collection 생성
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
    )

    # VectorStore 생성
    vectorstore = QdrantVectorStore(
        client=qdrant_client,
        collection_name=collection_name,
        embedding=embeddings,
    )

    # 문서 추가
    vectorstore.add_documents(documents)

    print(
        f"✓ VectorStore '{collection_name}' created: {len(documents)} documents indexed"
    )
    return vectorstore


def create_base_retriever(vectorstore: VectorStore, k: int = 5):
    """기본 similarity retriever를 생성합니다."""
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": k},
    )
    print(f"✓ Retriever created: top_k={k}")
    return retriever

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

# Load the document from docs/langchain_langgraph_full.md
loader = TextLoader("docs/langchain_langgraph_full.md", encoding="utf-8")
docs_list = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) # 청킹 전략
doc_splits = text_splitter.split_documents(docs_list)

# 벡터 스토어 생성 (Qdrant 사용 - 전체 노트북에서 재사용)
vectorstore = setup_vectorstore(documents=doc_splits, collection_name="langchain_docs")
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

print(f"✓ 문서 로드 완료: {len(docs_list)}개 원본 문서")
print(f"✓ 문서 분할 완료: {len(doc_splits)}개 청크")

# Define state schemas

In [ ]:
"""
Adaptive RAG State 및 분류 스키마
"""
from typing import Literal, Sequence, Annotated
from langchain.messages import AnyMessage
from langgraph.graph.message import add_messages
from langchain.tools import tool, ToolRuntime  # 새로운 tool, ToolRuntime 방식
from langchain.agents import create_agent
from pydantic import BaseModel, Field
from typing import TypedDict


class AdaptiveRAGState(TypedDict):
    """Adaptive RAG 워크플로우 State 정의."""

    messages: Annotated[Sequence[AnyMessage], add_messages]
    question: str
    query_type: Literal["vectorstore", "web_search", "direct_answer"]
    documents: Sequence[Document]
    answer: str


class QueryClassification(BaseModel):
    """
    Structured output schema for classifying how a user question should be handled.

    Purpose
        - Normalize the router’s decision into a stable `query_type`.
        - Provide a brief, auditable justification that downstream components can log or display.

    Fields
        - query_type (Literal["vectorstore", "web_search", "direct_answer"])
        Routing decision indicating whether to use internal documents, perform a live web search, or answer directly without retrieval.
        Allowed values (lowercase, exact match):
            - vectorstore — Use internal/embedded corpora (e.g., product/internal docs, API references, tutorials, design notes).
            - web_search — Use public web for time-sensitive or external information (e.g., news, release notes, pricing/policy changes).
            - direct_answer — Provide an answer directly without retrieval when the question is self-contained or solvable by reasoning.
        - reasoning (str)
        A concise justification (1–3 sentences) summarizing the key signals that led to the classification
        (e.g., intent, authority needs, recency requirements, specificity). Avoid chain-of-thought; state conclusions only.

    Decision Policy (guidelines, not enforced)
        - Prefer vectorstore for product/library usage, APIs, internal concepts, and stable specifications.
        - Prefer web_search for recent events, releases, pricing/policy/security updates, or content absent from internal corpora.
        - Prefer direct_answer for general knowledge, trivial lookups, or logical/derivational tasks that need no external facts.
    """

    query_type: Literal["vectorstore", "web_search", "direct_answer"] = Field(
        description=(
            "Routing decision for how to answer the question. "
            "'vectorstore': internal/embedded docs (APIs, concepts, tutorials). "
            "'web_search': public web for timely/external info (news, releases, pricing/policies). "
            "'direct_answer': answer without retrieval when self-contained or solvable by reasoning."
        )
    )
    # 실전에서는 사용하시는걸 비추천합니다. -> 출력이 길어질수록 느려집니다.
    # 개발할 때는 유용, 어떤 근거를 가지고 분류를 했는지 일치도 확인이 가능합니다.
    reasoning: str = Field(
        description=(
            "Short justification (1–3 sentences) explaining the classification. "
            "Mention key signals such as intent, timeliness/recency needs, specificity, and authority requirements. "
            "Avoid exposing chain-of-thought; provide summarized rationale only."
        )
    )


CLASSIFICATION_SYSTEM_PROMPT = """
You are an expert Query Classifier for a Retrieval-Augmented Generation (RAG) system.

[Task]
Analyze a single user message and assign exactly one category — `vectorstore, web_search, or direct_answer` — and provide a concise justification.

Use the following category definitions (placeholders will be provided at runtime):

1) vectorstore: Questions about {vectorstore_topics}, including synonyms, paraphrases, subtopics, implicit references, and project/product-specific terminology that is likely covered by the vector store.
2) web_search: Questions that meet {web_search_criteria}, typically requiring up-to-date, external, or not-in-vectorstore information, or when the user explicitly requests web sources.
3) direct_answer: Questions that meet {direct_answer_criteria}, solvable via general reasoning, known facts that do not require retrieval, or text manipulation of content provided directly by the user.

[Decision policy (apply in order)]

- If the query falls within or is plausibly about {vectorstore_topics}, choose vectorstore (even if the answer seems “easy” to the model).
- Else, if the query requires current/external information or explicitly asks to browse/search/cite the web per {web_search_criteria}, choose web_search.
- Else, choose direct_answer per {direct_answer_criteria}.

[Signals and heuristics]

Prefer `vectorstore` when:
The query targets proprietary/internal content, project artifacts, product features, code/docs known to be embedded, or asks to “use our docs.”
The user expects citations from internal docs or mentions internal identifiers, datasets, repositories, org-specific acronyms, or feature names tied to {vectorstore_topics}.

Prefer `web_search` when:
The query is time-sensitive (“latest”, “today”, “this year”), mentions news/markets/trends, or asks for statistics subject to change.
The topic is outside {vectorstore_topics}, references external entities/sites, or requests verification from multiple sources.
The user explicitly requests to “search the web,” “browse,” “open this link,” or “find current sources.”

Prefer `direct_answer` when:
The task is self-contained (math, logic, coding patterns without external APIs/docs, translation, summarization/rewrite of text provided in the message, grammar/style help).
The query asks for general principles or definitions that do not depend on internal or current information and do not require citations.

[Edge cases]
- Ambiguous/underspecified queries: Classify based on the most likely intent using the above policy; note what is missing in the reason.
- Multi-intent queries: Choose the dominant intent. If any major part is in-scope for {vectorstore_topics}, prefer vectorstore; otherwise, if any major part requires external/current info, prefer web_search; else direct_answer.
- Safety/Compliance content: Still classify; downstream policies handle refusals or safe-completions.
- Language: Classify regardless of user language. Reason should be in concise English.

[Constraints]
- Do not browse, search, or retrieve content. Only classify.
- Be precise and avoid speculation beyond the signals present in the query.
- Use short, explicit rationales referencing the key signals (e.g., “mentions internal feature X,” “needs current stats,” “simple arithmetic”).

[Output format] (return only this JSON object, no extra text):
{{"category":"vectorstore|web_search|direct_answer","reason":"<one or two concise sentences be shortend>"}}
"""

VECTORSTORE_TOPICS = (
    "LangChain, LangGraph, Multi-agent 시스템, RAG, 벡터 검색, 기술 문서"
)
WEB_SEARCH_CRITERIA = "최신 뉴스, 실시간 정보 등 웹 검색"
DIRECT_ANSWER_CRITERIA = "일반 지식, 수학 계산 또는 검색 없이 답변 가능한 사실적 질문"

## Define Tools

In [ ]:
"""
도구 정의

이 셀에서 정의된 도구들은 노트북 전체에서 재사용됩니다:
- search_vectorstore: 벡터 DB 검색
- search_web: 웹 검색 (Tavily)
- calculate: 계산기
- tools: 도구 리스트
"""
from langchain_tavily import TavilySearch
import ast
import operator as op
from typing import Any

# ========================================
# Tool 1: Vectorstore 검색
# ========================================

@tool
def search_vectorstore(query: str) -> str:
    """
    벡터 데이터베이스에서 관련 문서를 검색합니다.

    Args:
        query: 검색 쿼리

    Returns:
        검색된 문서 내용
    """
    if "vectorstore" not in globals() or vectorstore is None:
        return " vectorstore가 정의되지 않았습니다. Section 1.2의 문서 로드 셀을 먼저 실행하세요."

    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    docs = retriever.invoke(query)

    if not docs:
        return "관련 문서를 찾지 못했습니다."

    result = "검색 결과:\n\n"
    for i, doc in enumerate(docs, 1):
        result += f"문서 {i}:\n{doc.page_content}\n\n"
    return result


# ========================================
# Tool 2: 웹 검색 (Tavily API)
# ========================================

search_web = TavilySearch()

# ========================================
# Tool 3: 계산기
# ========================================

# 안전한 수식 평가를 위한 허용 연산자
_ALLOWED_OPERATORS: dict[type[ast.AST], Any] = {
    ast.Add: op.add,
    ast.Sub: op.sub,
    ast.Mult: op.mul,
    ast.Div: op.truediv,
    ast.Pow: op.pow,
    ast.USub: op.neg,
}


def _eval_expr(expr: ast.Expression) -> float:
    """안전하게 수식을 평가합니다."""
    return _eval(expr.body)


def _eval(node: ast.AST) -> float:
    """AST 노드를 재귀적으로 평가합니다."""
    if isinstance(node, ast.Constant):
        return float(node.value)
    elif isinstance(node, ast.BinOp):
        return _ALLOWED_OPERATORS[type(node.op)](_eval(node.left), _eval(node.right))
    elif isinstance(node, ast.UnaryOp):
        return _ALLOWED_OPERATORS[type(node.op)](_eval(node.operand))
    else:
        raise TypeError(f"Unsupported operation: {node}")


@tool
def calculate(expression: str) -> str:
    """
    수학 계산을 수행합니다.

    Args:
        expression: 계산할 수식 (예: "2 + 3 * 4")

    Returns:
        계산 결과
    """
    try:
        tree = ast.parse(expression, mode="eval")
        result = _eval_expr(tree)
        return f"{expression} = {result}"
    except Exception as e:
        return f"계산 오류: {e}"


# ========================================
# 도구 리스트
# ========================================

tools = [search_vectorstore, search_web, calculate]
tools

# Define Nodes, Edges

In [ ]:
"""
Adaptive RAG 노드 및 라우팅 로직 (Part 1)
"""

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.documents import Document

# ========================================
# 노드 함수 (Part 1: Classification & Retrieval)
# ========================================


def retrieve_documents(state: AdaptiveRAGState) -> dict:
    """벡터 스토어에서 관련 문서를 검색합니다."""
    question = state["question"]
    # retriever를 직접 사용하여 Document 객체 리스트를 반환
    docs = retriever.invoke(question)

    return {"documents": docs}


def web_search_node(state: AdaptiveRAGState) -> dict:
    """웹 검색을 통해 최신 정보를 가져옵니다."""
    question = state["question"]
    result = search_web.invoke({"query": question})

    docs = [Document(page_content=result)] if result else []
    return {"documents": docs}


def classify_query(state: AdaptiveRAGState) -> dict:
    """사용자 질문을 분류하여 후속 라우팅에 사용합니다."""
    question = state["question"]

    system_prompt = CLASSIFICATION_SYSTEM_PROMPT.format(
        vectorstore_topics=VECTORSTORE_TOPICS,
        web_search_criteria=WEB_SEARCH_CRITERIA,
        direct_answer_criteria=DIRECT_ANSWER_CRITERIA,
    )

    classification: QueryClassification = classifier_llm.invoke(
        [
            SystemMessage(content=system_prompt),
            HumanMessage(content=question),
        ]
    )

    # Adaptive RAG에서는 명시적으로 상태를 업데이트하여 다음 노드의 입력을 제어합니다.
    return {"query_type": classification.query_type, "documents": []}


def generate_answer(state: AdaptiveRAGState) -> dict:
    """검색된 컨텍스트 또는 내부 지식을 기반으로 최종 답변을 생성합니다."""
    question = state["question"]
    query_type = state.get("query_type", "vectorstore")
    documents = state.get("documents", [])

    if query_type == "direct_answer":
        direct_response = llm.invoke(
            [
                SystemMessage(
                    content="""
                [Role]
                You are an expert assistant. 
                Provide concise, well‑supported answers in Korean.

                [Objectives]
                - Deliver a correct, succinct answer with essential rationale.
                - Summarize the answer with brief bullet points when helpful.
                - Always end with the exact line: 출처: 내부 지식

                [Language]
                - Write the final answer in Korean (formal, professional tone).
                - Keep technical terms, APIs, and proper nouns in English when clearer; add Korean explanations if needed.

                [Answering Policy]
                - Be precise and avoid speculation. If information is missing or ambiguous, ask 1–3 focused clarifying questions in Korean before proceeding, or clearly state minimal assumptions.
                - Prefer actionable guidance (steps, key commands, or code snippets) when relevant; keep examples minimal and directly tied to the question.
                - Do not reveal internal reasoning or chain‑of‑thought; present conclusions and key evidence only.

                [Safety and Neutrality]
                - Remain neutral, professional, and respectful.
                - Flag potential risks, limitations, or prerequisites succinctly when material to correct use.

                [Output Format]
                - 답변: 2–6 sentences directly addressing the question.
                - 요약: 2–5 bullets highlighting the key takeaways.
                - 출처: 내부 지식

                [Style Guidelines]
                - Keep it concise; avoid redundancy.
                - Use clear headings and bullets sparingly for readability.
                - Use consistent terminology; include versions/dates if relevant to accuracy.

                [Examples (format only)]
                답변: 해당 기능은 서버 측에서 토큰 단위로 처리되며, 비동기 스트리밍을 통해 클라이언트에 전달됩니다. 설정에서 이벤트 스트림을 활성화하고, 각 청크에 대한 누적 토큰 수를 로깅하는 것이 권장됩니다.
                요약:
                - 서버 비동기 스트리밍 기반 동작
                - 설정에서 이벤트 스트림 활성화 필요
                - 청크별 누적 토큰 로깅 권장
                출처: 내부 지식
                """
                ),
                HumanMessage(content=question),
            ]
        )
        return {"answer": direct_response.content}

    context = "\n\n".join([doc.page_content for doc in documents]) if documents else ""

    if not context.strip():
        user_prompt = f"""
            [Status]
            No external context was provided.

            [Request]
            Do not guess. Specify the additional information required to answer accurately. Respond in English. Use Markdown.

            [Question]
            {question}

            [Output Format]
            - Information Request: 3–8 bullets listing specific missing details
            - Minimum Required: 1–2 lines stating the minimal context needed to proceed
            - Example (optional): one line showing an example of a helpful input format

            [Constraints]
            - Return only the sections above; do not attempt to answer the question itself.
        """
    else:
        user_prompt = f"""
            [Instruction]
            You will receive "Context Sources" and a "Question." Follow the System Prompt strictly. Respond in English with inline numeric citations and a final "Sources" section. Use Markdown.

            [Context Sources]
            {context}

            [Question]
            {question}

            [Output Requirements]
            - Follow the required format defined in the System Prompt exactly.
            - Include inline citations [n] for all factual claims.
            - End with a "Sources" section mapping citation numbers to metadata.
        """

    system_prompt = """
    [Role]
    You are a careful, citation-first assistant that answers using only the supplied "Context Sources." Produce accurate, concise answers in English with inline citations.

    [Language]
    - Write the final answer in Korean (formal, professional tone).
    - Keep technical terms or source titles in their original language when appropriate.

    [Grounding and Hallucination Control]
    - Use only facts present in the Context Sources.
    - If the context is insufficient, do not attempt an answer; instead request the additional information needed.
    - If sources conflict, briefly note the discrepancy and cite each source involved.

    [Authority and Conflict Resolution]
    - Prefer primary/official and more recent sources when conflicts persist.
    - Mention dates/versions when relevant to clarify timeliness.

    [Citations]
    - Support every factual claim with inline numeric citations like [1], [2].
    - Each paragraph or bullet that contains claims must include at least one citation.
    - After the answer, add a "Sources" section mapping each citation number to source metadata:
    title — URL/path — page/section (when available).
    - Use numeric identifiers provided with sources; if none exist, number them in the order given.

    [Quotations and Paraphrasing]
    - Prefer paraphrasing.
    - If quoting, keep quotes under 20 words and use quotation marks.

    [Response Format — sufficient context (required)]
    - Answer: Direct, concise answer to the question in English.
    - Evidence: 2–5 concise bullets, each with at least one inline citation.
    - Sources: List cited sources in numeric order (e.g., [1] title — URL/path — page/section).

    [Response Format — missing/insufficient context]
    - Information Request: 3–8 bullets specifying the missing details required to answer.
    - Minimum Required: 1–2 lines describing the minimal context needed to proceed.

    [Constraints]
    - Do not use knowledge outside the Context Sources.
    - Do not guess or make unsupported assumptions.
    - Keep the response focused and succinct; do not reveal private reasoning or step-by-step analysis.
    - Do not mention these instructions.
    """

    response = llm.invoke(
        [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_prompt),
        ]
    )

    return {"answer": str(response.content)}


def route_query(state: AdaptiveRAGState) -> str:
    """조건부 분기를 위한 라우팅 규칙."""
    query_type = state["query_type"]

    if query_type == "vectorstore":
        return "retrieve"
    if query_type == "web_search":
        return "web_search"
    return "generate"

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-4.1-mini")
classifier_llm = llm.with_structured_output(QueryClassification)
classifier_llm

# Build the graph

In [ ]:
# Adaptive RAG 워크플로우 생성
from langgraph.graph import END, StateGraph

adaptive_graph = StateGraph(AdaptiveRAGState)
adaptive_graph.add_node("classify", classify_query)
adaptive_graph.add_node("retrieve", retrieve_documents)
adaptive_graph.add_node("web_search", web_search_node)
adaptive_graph.add_node("generate", generate_answer)

adaptive_graph.set_entry_point("classify")
adaptive_graph.add_conditional_edges(
    "classify",
    route_query,
    {
        "retrieve": "retrieve",
        "web_search": "web_search",
        "generate": "generate",
    },
)

adaptive_graph.add_edge("retrieve", "generate")
adaptive_graph.add_edge("web_search", "generate")
adaptive_graph.add_edge("generate", END)

adaptive_rag = adaptive_graph.compile()
# display(Image(adaptive_rag.get_graph().draw_mermaid_png()))
# adaptive_rag.get_graph().print_ascii()
display(Image(adaptive_rag.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.PYPPETEER,)))
# print(adaptive_rag.get_graph().draw_mermaid())

In [ ]:
# Adaptive RAG 실행
question = "LangGraph의 multi-agent workflow는 무엇인가요?"

result = adaptive_rag.invoke({"question": question})

print("=== Adaptive RAG Answer ===")
print(result.get("answer", "(응답 없음)"))

In [ ]:
# Adaptive RAG 실행
question = "LangGraph에서 Tool 사용법을 알려주세요."

result = adaptive_rag.invoke({"question": question})

print("=== Adaptive RAG Answer ===")
print(result.get("answer", "(응답 없음)"))

In [ ]:
# Adaptive RAG 실행
question = "페이커 근황을 알려주세요.."

result = adaptive_rag.invoke({"question": question})

print("=== Adaptive RAG Answer ===")
print(result.get("answer", "(응답 없음)"))

In [ ]:
"""
인용 번호를 하이퍼링크로 변환하는 후처리 함수
"""
import re
from IPython.display import Markdown, display


def add_citation_links(answer_text: str) -> str:
    """
    답변 텍스트에서 [1], [2] 같은 인용 번호를 클릭 가능한 하이퍼링크로 변환합니다.
    
    Args:
        answer_text: LLM이 생성한 답변 텍스트 (Sources 섹션 포함)
    
    Returns:
        Markdown 형식의 하이퍼링크가 포함된 텍스트
    
    Example:
        Input:  "LangGraph는 노드로 구성됩니다[1]. 상태를 공유합니다[2]."
        Output: "LangGraph는 노드로 구성됩니다[[1]](#source-1). 상태를 공유합니다[[2]](#source-2)."
    """
    # 단계 1: Sources 섹션에 anchor 추가
    def add_source_anchors(match):
        """Sources 섹션의 각 항목에 HTML anchor 추가"""
        num = match.group(1)
        rest = match.group(2)
        return f'<a id="source-{num}"></a>[{num}]{rest}'
    
    # Sources 섹션의 [1], [2] 등에 anchor 추가
    # 패턴: 줄 시작 + [숫자] + 공백 또는 나머지 텍스트
    text_with_anchors = re.sub(
        r'^(\[\d+\])(\s.*)$',
        add_source_anchors,
        answer_text,
        flags=re.MULTILINE
    )
    
    # 단계 2: 본문의 인용 번호를 하이퍼링크로 변환
    def convert_citation_to_link(match):
        """본문의 [1] → [[1]](#source-1) 형식으로 변환"""
        num = match.group(1)
        return f'[[{num}]](#source-{num})'
    
    # Sources 섹션 이전의 본문에서만 변환
    # (Sources 섹션 자체는 이미 anchor가 있으므로 제외)
    parts = text_with_anchors.split('Sources:', 1)
    
    if len(parts) == 2:
        # Sources 이전 본문만 링크 변환
        body = parts[0]
        sources_section = parts[1]
        
        # 본문의 [숫자] 패턴을 하이퍼링크로 변환
        body_with_links = re.sub(r'\[(\d+)\]', convert_citation_to_link, body)
        
        # 재조합
        result = body_with_links + 'Sources:' + sources_section
    else:
        # Sources 섹션이 없으면 전체 변환
        result = re.sub(r'\[(\d+)\]', convert_citation_to_link, text_with_anchors)
    
    return result


# 테스트
sample_answer = """
LangGraph는 노드로 구성됩니다[1]. 상태를 공유합니다[2].

Sources:
[1] LangGraph documentation — /oss/langgraph
[2] Tutorial — /oss/tutorial
"""

print("=== 원본 ===")
print(sample_answer)

print("\n=== 변환 결과 ===")
converted = add_citation_links(sample_answer)
print(converted)

print("\n=== Markdown 렌더링 (Jupyter에서만 작동) ===")
display(Markdown(converted))


In [ ]:
# 실제 Adaptive RAG 답변에 적용
answer_with_links = add_citation_links(result.get("answer", ""))

print("=== 하이퍼링크가 추가된 답변 (Markdown) ===\n")
display(Markdown(answer_with_links))
